In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import os

def get_glove_vectors(filename="data/glove.6B.100d.txt"):
    ## function from https://campus.datacamp.com/courses/recurrent-neural-networks-for-language-modeling-in-python/rnn-architecture?ex=7
    # Get all word vectors from pre-trained model
    glove_vector_dict = {}
    with open(filename, encoding="UTF-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = values[1:]
            glove_vector_dict[word] = np.asarray(coefs, dtype='float32')
    return glove_vector_dict

In [2]:
print(tf.__version__)
print(keras.__version__)

2.9.1
2.9.0


In [3]:
import time
start = time.time()

glove_vector_dict = get_glove_vectors()

end = time.time()
print(f'elapsed seconds = {end - start}')
type(glove_vector_dict)

elapsed seconds = 10.03065276145935


dict

In [4]:
glove_vector_dict['the']

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [5]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
## Previous models ignored the keyword column in the training and test data.

## To incorporate the keyword, we will try just appending the keywords (when present) as
## an additional one or two tokens at the end of each tweet text.

## This function modifies the pandas dataframe df by
## appending the keyword (if present) to the end of the tweet.
## Keywords like 'airplane%20accident' are split into two words.
## It also writes the modified dataframe to a csv file (for debugging).
def add_keyword(df, filename=None):
    df.loc[df['keyword'].isna()==False,'text'] = df['text'] + ' ' + df['keyword'].str.replace('%20',' ')
    if filename:
        df.to_csv(f'data/{filename}',index=False)

add_keyword(df_train, filename='df_train.csv')
add_keyword(df_test, filename='df_test.csv')

In [7]:
s1 = df_train[df_train['id']==232]
print(s1)
s2 = df_train.iloc[161,3]
s2

      id              keyword   location  \
161  232  airplane%20accident  Havenford   

                                                  text  target  
161  + Nicole Fletcher one of a victim of crashed a...       1  


"+ Nicole Fletcher one of a victim of crashed airplane few times ago. \n\nThe accident left a little bit trauma for her. Although she's \n\n+ airplane accident"

In [8]:
df_train['target'].agg(sum)/len(df_train)

0.4296597924602653

In [9]:
import re

def clean_up_tweet(tweet):
    """
    Clean up the content of one tweet, removing punctuation and numbers. 
    
    Parameters:
    tweet(str):The text of the tweet
    
    Returns:
    word_list: A list of pure alphabetic words in lower case
    
    """
    ## Remove all characters execept alphabetic chars and space,
    ## convert to lower case and split on space.
    word_list = re.sub('[^A-Za-z ]+','',tweet).lower().split(' ')
    return word_list
    

In [10]:
clean_up_tweet(s2)

['',
 'nicole',
 'fletcher',
 'one',
 'of',
 'a',
 'victim',
 'of',
 'crashed',
 'airplane',
 'few',
 'times',
 'ago',
 'the',
 'accident',
 'left',
 'a',
 'little',
 'bit',
 'trauma',
 'for',
 'her',
 'although',
 'shes',
 '',
 'airplane',
 'accident']

In [11]:
train, valid = train_test_split(df_train, train_size=0.8, shuffle=True, random_state=42)
print(train.shape)
print(valid.shape)

(6090, 5)
(1523, 5)


In [12]:
train_x = train['text'].map(clean_up_tweet)
valid_x = valid['text'].map(clean_up_tweet)
test_x = df_test['text'].map(clean_up_tweet)

print(train_x.shape)
print(valid_x.shape)
print(test_x.shape)
type(train_x)

(6090,)
(1523,)
(3263,)


pandas.core.series.Series

In [13]:
train_y = np.array(train['target'], dtype=np.float32)
valid_y = np.array(valid['target'], dtype=np.float32)
print(train_y[:5])
print(train_y[-5:])
print(np.sum(train_y)/len(train_y))
print(np.sum(valid_y)/len(valid_y))

[1. 0. 1. 1. 0.]
[0. 0. 0. 1. 1.]
0.43054187192118226
0.4261326329612607


In [14]:
train['tweet_word_counts'] = [len(x) for x in train_x]
valid['tweet_word_counts'] = [len(x) for x in valid_x]
df_test['tweet_word_counts'] = [len(x) for x in test_x]
print(np.max(train['tweet_word_counts']) )
print(np.max(valid['tweet_word_counts']) )
print(np.max(df_test['tweet_word_counts']) )

55
31
35


In [15]:
def glove_word_embeddings(word_lists, pad_to=56):
    ## We plan to replace all the words in the tweets
    ## with embeddings from the GloVe dictionary, skipping
    ## any words not found, and also padding the sequence 
    ## of embeddings to a fixed length.
    
    ## If none of the words match for a given tweet we will substitute
    ## a with place holder vector of one word, "neutral".
    d = glove_vector_dict
    neutral = d["neutral"]
    placeHolder = np.array([neutral])
    padNeutral = pad_sequences(placeHolder.T, pad_to, dtype='float32')
    ## print('padNeutral',padNeutral)
    outer = []
    for word_list in word_lists:
        enc_list = []
        for word in word_list:
            ## print(word)
            if(type(d.get(word)) is np.ndarray):
                enc_list.append(d.get(word))
                ## print(d.get(word))
        if(len(enc_list) > 0):
            enc_array = np.array(enc_list)
#             print('shape: ',enc_array.shape)
#             print('enc_array = ',enc_array)
#             print('enc_array.T', enc_array.T)
            pad = pad_sequences(enc_array.T, pad_to, dtype='float32')
#             print('pad',pad)
            outer.append(pad.T)
#             print('outer shape',outer.shape)
#             print('outer', outer)
        else:
            outer.append(padNeutral.T)
    return np.array(outer)

In [16]:
print (train_x.shape)
print (valid_x.shape)
print (test_x.shape)

(6090,)
(1523,)
(3263,)


In [17]:

start = time.time()
X_train = glove_word_embeddings(train_x)
X_valid = glove_word_embeddings(valid_x)
X_test = glove_word_embeddings(test_x)
end = time.time()
print(f'elapsed seconds = {end - start}')
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

elapsed seconds = 2.0369720458984375
(6090, 56, 100)
(1523, 56, 100)
(3263, 56, 100)


In [18]:
DROPOUT = 0.3
UNITS_PER_LAYER = 64

## Try switching to a Bidirectional LSTM model, as in this example
## https://keras.io/examples/nlp/bidirectional_lstm_imdb/

from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(None, 100) )
x = layers.Bidirectional(LSTM(units=UNITS_PER_LAYER, return_sequences=True, dropout=DROPOUT))(inputs)
x = layers.Bidirectional(LSTM(units=UNITS_PER_LAYER, return_sequences=True, dropout=DROPOUT))(x)
x = layers.Bidirectional(LSTM(units=UNITS_PER_LAYER, return_sequences=False, dropout=DROPOUT))(x)
# Add a classifier
outputs = layers.Dense(1,  activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

opt = tf.keras.optimizers.Adam(learning_rate=0.0003)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

file_name = 'weights_{epoch:03d}_{val_accuracy:.4f}.hdf5'

checkpoint_filepath = os.path.join('.', 'SAVE_MODELS', file_name)

modelCheckpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

earlyStopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6, restore_best_weights=True)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 100)]       0         
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        84480     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 128)        98816     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                             

In [19]:
history = model.fit(X_train, train_y, 
                    batch_size=20, 
                    epochs=100, 
                    validation_data=(X_valid,valid_y),
                    callbacks=[earlyStopping,modelCheckpoint]
                   )

Epoch 1/100
305/305 [==============================] - 20s 38ms/step - loss: 0.4999 - accuracy: 0.7677 - val_loss: 0.4307 - val_accuracy: 0.8234
Epoch 2/100
305/305 [==============================] - 9s 31ms/step - loss: 0.4402 - accuracy: 0.7970 - val_loss: 0.4229 - val_accuracy: 0.8168
Epoch 3/100
305/305 [==============================] - 9s 31ms/step - loss: 0.4345 - accuracy: 0.8095 - val_loss: 0.4160 - val_accuracy: 0.8326
Epoch 4/100
305/305 [==============================] - 9s 31ms/step - loss: 0.4251 - accuracy: 0.8115 - val_loss: 0.4157 - val_accuracy: 0.8227
Epoch 5/100
305/305 [==============================] - 9s 31ms/step - loss: 0.4149 - accuracy: 0.8151 - val_loss: 0.4213 - val_accuracy: 0.8253
Epoch 6/100
305/305 [==============================] - 9s 31ms/step - loss: 0.4092 - accuracy: 0.8163 - val_loss: 0.4316 - val_accuracy: 0.8148
Epoch 7/100
305/305 [==============================] - 9s 31ms/step - loss: 0.4040 - accuracy: 0.8238 - val_loss: 0.4092 - val_accuracy

In [20]:
model.evaluate(X_valid, valid_y)

48/48 [==============================] - 1s 18ms/step - loss: 0.4092 - accuracy: 0.8273


[0.4091683030128479, 0.8273144960403442]

In [22]:
model.evaluate(X_valid, valid_y)

48/48 [==============================] - 1s 16ms/step - loss: 0.4092 - accuracy: 0.8273


[0.4091683030128479, 0.8273144960403442]

In [68]:
## Set aside full data set for retraining for competition
## see if training on the full training set improves results

full_x = df_train['text'].map(clean_up_tweet)
full_y = np.array(df_train['target'], dtype=np.float32)
X_full = glove_word_embeddings(full_x)


In [69]:
history = model.fit(X_full, full_y, 
                    batch_size=20, 
                    epochs=3, 
                   )

Epoch 1/3
381/381 [==============================] - 10s 19ms/step - loss: 0.4882 - accuracy: 0.7818
Epoch 2/3
381/381 [==============================] - 7s 19ms/step - loss: 0.4380 - accuracy: 0.8078
Epoch 3/3
381/381 [==============================] - 7s 19ms/step - loss: 0.4199 - accuracy: 0.8172


In [23]:
sub_num = 12

predict_proba = model.predict(X_test,batch_size=20)
predict = (predict_proba > 0.5).astype(int)

submission = pd.DataFrame(df_test['id'])
submission['target']=predict

submission.to_csv(f'data/submission{sub_num}.csv',index=False)
submission
model.evaluate(X_valid, valid_y)

48/48 [==============================] - 1s 25ms/step - loss: 0.4092 - accuracy: 0.8273


[0.4091683030128479, 0.8273144960403442]